          Classification avec les donnees Titanic:

importation des differents packages:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from yellowbrick.classifier import (
    ConfusionMatrix,
    ROCAUC,
)

from yellowbrick.model_selection import (
    LearningCurve,
)
from sklearn import (
    ensemble,
    preprocessing,
    tree,
)

from sklearn.metrics import (
    auc,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
)

from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
)



Poser les termes de la question:

Dans l'exemple, nous voulons obtenir un modéle prédictif qui va répondre à une question.Nous allons créer une classification
pour deviner les chances de survie d'un individu dans la catastrophe du Titanic en nous basant sur des caractéristiques
de cette personne et de ses conditions de présence à bord.
Notre modele va exploiter les informations des passagers pour predire la survie d'un passager du naufrage du Titanic.
Il s'agit ici d'un projet de classification,puisque nous allons prédire des noms (de labels) pour la survie: soit la personne survit,
soit elle décède.

In [3]:
# charger les donnees et afficher les 5 premieres lignes
df = pd.read_csv('tested.csv')
print(df.head(5))

# les differesnts colonnes de notre dataframe
print(list(df.columns))

   PassengerId  Survived  Pclass  \
0          892         0       3   
1          893         1       3   
2          894         0       2   
3          895         0       3   
4          896         1       3   

                                           Name     Sex   Age  SibSp  Parch  \
0                              Kelly, Mr. James    male  34.5      0      0   
1              Wilkes, Mrs. James (Ellen Needs)  female  47.0      1      0   
2                     Myles, Mr. Thomas Francis    male  62.0      0      0   
3                              Wirz, Mr. Albert    male  27.0      0      0   
4  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female  22.0      1      1   

    Ticket     Fare Cabin Embarked  
0   330911   7.8292   NaN        Q  
1   363272   7.0000   NaN        S  
2   240276   9.6875   NaN        Q  
3   315154   8.6625   NaN        S  
4  3101298  12.2875   NaN        S  
['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket'

Nettoyage des données:

In [4]:
# les types de donnees 
print(df.dtypes)



PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


In [5]:
# les nombres de lignes et colonnes de notre dataframe
print(df.shape)

(418, 12)


In [6]:
# la methode describe() nous permet d'obtenir des stat
print(df.describe())


       PassengerId    Survived      Pclass         Age       SibSp  \
count   418.000000  418.000000  418.000000  332.000000  418.000000   
mean   1100.500000    0.363636    2.265550   30.272590    0.447368   
std     120.810458    0.481622    0.841838   14.181209    0.896760   
min     892.000000    0.000000    1.000000    0.170000    0.000000   
25%     996.250000    0.000000    1.000000   21.000000    0.000000   
50%    1100.500000    0.000000    3.000000   27.000000    0.000000   
75%    1204.750000    1.000000    3.000000   39.000000    1.000000   
max    1309.000000    1.000000    3.000000   76.000000    8.000000   

            Parch        Fare  
count  418.000000  417.000000  
mean     0.392344   35.627188  
std      0.981429   55.907576  
min      0.000000    0.000000  
25%      0.000000    7.895800  
50%      0.000000   14.454200  
75%      0.000000   31.500000  
max      9.000000  512.329200  


In [17]:
# traiter les donnees manquantes,ici nous avons les nombres de donnees manquantes dans chaque colonne
print(df.isnull().sum())
#print(list(df.columns))

Survived     0
Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64


In [8]:
# voir les differents categories des variables categorielles
print(df['Sex'].value_counts(dropna=False))
print(df['Pclass'].value_counts(dropna=False))

Sex
male      266
female    152
Name: count, dtype: int64
Pclass
3    218
1    107
2     93
Name: count, dtype: int64


Creation des caracteristiques

In [21]:
# utilisons la methode drop() de pandas pour eliminer des colonnes
#df = df.drop(columns=['Name','Cabin','Ticket','PassengerId'],axis=1)

# creation des colonnes factices pour les variables categorielles
df = pd.get_dummies(df, drop_first=True)
print(list(df.columns))


['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S']


In [29]:
# supprimer les colonnes

#df = df.drop(columns='Sex_male')
#df = df.drop(columns='Embarked_C')
print(list(df.columns))

['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Embarked_Q', 'Embarked_S']


In [30]:
# creons une dataframe  X avec les caracteristiques et une serie y avec les labels
y = df['Survived']
X = df.drop(columns='Survived')

print(X.shape)
print(y.shape)

(418, 8)
(418,)


In [32]:
# selection d'un echantillons de donnees
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

(292, 8)
(292,)
(126, 8)
(126,)


Imputation de donnees: